# Exercise 5.2

First of all, we import usefull libraries.

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import cvxopt

Now, we define a function to choose set of training or testing data for data_set and labels, as well as another one to obtain length of labels array.

In [2]:
def choose_set_for_label(data_set, labels, i):
    data_set = data_set[labels!=i]
    labels = labels[labels!=i]
    
    labels[labels == np.min(labels)] = -1
    labels[labels == np.max(labels)] = 1
    
    positions_train = np.random.choice(range(len(labels)), int(len(labels) * alpha), replace = False)
    positions_test = np.delete(range(len(labels)), positions_train)
    # We're taking alpha fot train and (1-alpha) for test
    
    train_data_set = data_set[positions_train]
    train_labels = labels[positions_train]
    
    test_data_set = data_set[positions_test]
    test_labels = labels[positions_test]
    return train_data_set, test_data_set, train_labels, test_labels

def get_labels_count(data_set):
    labels_count = len(data_set)
    return labels_count

Use the code that we have implemented earlier:

In [3]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

We load the data and apply the functions we defined before. We run three times the code, so we obtain the predicted results from the three pairs. Then, we apply a voting method so decide the correct value from the three sets and, finally, obtain the accuracy.

We're iterating everything, but we may have a problem when we compare it with tsta_labels.

In [4]:
iris = load_iris()
data_set = iris.data
labels = iris.target

predictions = []

alpha = 0.85


for i in range(3):
    train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(data_set, labels, i)
    objects_count = get_labels_count(train_labels)
    
    lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf')
    predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
    predicted = list(predicted.astype(int))
    predictions.append(predicted)

nc, nf, o = np.shape(predictions)
predictions = np.reshape(predictions, (nf, nc))

min_value = np.min(predictions)
predictions -= min_value
final_predition = np.zeros(nf)

for i in range(nf):
    aux = [predictions[i, 0], predictions[i, 1], predictions[i, 2]]
    final_predition[i] = np.argmax(np.bincount(aux))

final_predition += min_value

#print('Obtained predition is:', final_predition)
print('The accuracy is:', accuracy_score(final_predition, test_labels))

     pcost       dcost       gap    pres   dres
 0:  8.3328e+01 -1.4131e+03  3e+03  2e-01  2e-15
 1:  5.9010e+01 -1.3139e+02  2e+02  6e-03  3e-15
 2:  7.9272e+00 -1.6941e+01  2e+01  6e-16  3e-15
 3: -3.1518e-01 -3.7574e+00  3e+00  2e-16  8e-16
 4: -1.0884e+00 -1.7738e+00  7e-01  2e-16  4e-16
 5: -1.2320e+00 -1.4432e+00  2e-01  2e-16  2e-16
 6: -1.3004e+00 -1.4231e+00  1e-01  2e-16  2e-16
 7: -1.3297e+00 -1.3658e+00  4e-02  2e-16  3e-16
 8: -1.3411e+00 -1.3436e+00  3e-03  2e-16  3e-16
 9: -1.3421e+00 -1.3423e+00  1e-04  2e-16  3e-16
10: -1.3422e+00 -1.3422e+00  1e-06  2e-16  2e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.1187e+02 -1.3598e+03  3e+03  2e-01  2e-15
 1:  6.6143e+01 -1.3711e+02  2e+02  5e-03  2e-15
 2:  7.8219e+00 -1.8856e+01  3e+01  5e-16  2e-15
 3: -7.2012e-01 -4.3668e+00  4e+00  2e-16  6e-16
 4: -1.4423e+00 -2.3749e+00  9e-01  2e-16  3e-16
 5: -1.7426e+00 -2.0671e+00  3e-01  4e-16  3e-16
 6: -1.8095e+00 -1.9085e+00  1e-01  2e-16  2e-1